In [79]:
#running all cells should result in one final file: 'combined.csv'
# report[year].csv --> clean1-[year].csv
import re
import pandas as pd

input_file_template = 'original-reports/report-'#<00,01,02,03,09...>.csv
output_file_template = 'cleaning/clean1-'

years = ['97','98','99','00','01','02','03','09','10','11','12','13','15']#match the end of files

for i in years:
    #print(i)
    in_file = "{}{}.csv".format(input_file_template,i)
    print('opening{}'.format(in_file))
    inf = open(in_file,'r')
    line_number = 0    
    state = 0
    year_string = ''
    column_headers = []
    out_file = "{}{}.csv".format(output_file_template,i)
    outf = open(out_file,'w+')
    for line in inf:
        line_number += 1

        
        #determine state
        prev_state = state
        if 'Population Estimates",,,"Delinquency",,"Status",,"Dependency",,' in line:
            state=1
        elif 'Reporting Counties' in line and state==1:
            state=2
        elif 'Number of Reported Cases' in line and state==2:
            state=3
        elif 'Population Represented' in line and state==3:
            state=4
        elif 'Rates for Reporting Counties' in line and state==4:
            state=5
        elif 'Number of Reporting Counties' in line and state==5:
            state=6
        elif len(re.findall(r'^\"\*Denotes',line))==1 and state==6:
            state=7
        
        #state work
        #print('[{}]{}:{}'.format(state,line_number,line))
        if state==1:#extract year
            year = re.search(r'([0-9]{4})',line)
            year_string = line.split(',')[1]
            print('found year={}'.format(year_string))
        elif prev_state==1 and state==2:#parse headers
            column_headers = re.sub(r'"','',line).strip().split(',')
            print('found column Headers: {}'.format(column_headers))
            #continue
#         elif state==2:
#             continue
        elif line is None:
            state=99
        
        if state==1:
            out_string = 'Year,Reporting Counties,{},{},{},Delinquency,Delinquency,Status,Status,Dependency,Dependency\n'.format(
                year_string,year_string,year_string)
            outf.write(out_string)
            print(out_string)
        elif state==2 and 'Reporting Counties' in line:
            out_string = 'Year,County Name,{}'.format(','.join(line.split(',')[1:-1]))
            outf.write(out_string+'\n')
            print(out_string)
        elif state==2:
            out_string=','.join(line.strip().split(',')[:-1])
            outf.write(year.group(0)+','+out_string+'\n')
            print(out_string)
    outf.close()
    inf.close()

openingoriginal-reports/report-97.csv
found year="1997 Population Estimates"
Year,Reporting Counties,"1997 Population Estimates","1997 Population Estimates","1997 Population Estimates",Delinquency,Delinquency,Status,Status,Dependency,Dependency

found column Headers: ['Reporting Counties', 'Total', '10 Through Upper Age', '0 Through Upper Age', 'Petition', 'Non-petition', 'Petition', 'Non-petition', 'Petition', 'Non-petition', 'All Reported Cases']
Year,County Name,"Total","10 Through Upper Age","0 Through Upper Age","Petition","Non-petition","Petition","Non-petition","Petition","Non-petition"
"Albany","29,700","2,700","5,800","45","--","10","--","17","--"
"Big Horn","11,000","1,600","3,000","45","--","*","--","*","--"
"Campbell","32,100","5,400","10,800","65","--","11","--","10","--"
"Carbon","15,800","2,300","4,300","36","--","9","--","12","--"
"Converse","12,300","2,000","3,700","25","--","*","--","0","--"
"Crook","5,800","900","1,700","18","--","*","--","8","--"
"Fremont","35,900",

In [84]:
#clean1-[year].csv -> combined.csv
input_file_template = 'cleaning/clean1-'#<00,01,02,03,09...>.csv
output_file_template = 'combined.csv'

outf = open(output_file_template,'w+')
outf.write('Year,Reporting Counties,Population Estimates,Population Estimates,Population Estimates,Delinquency,Delinquency,Status,Status,Dependency,Dependency\n')
#Year,Reporting Counties,"Population Estimates","Population Estimates","Population Estimates",Delinquency,Delinquency,Status,Status,Dependency,Dependency
outf.write('Year,County Name,Total,10 Through Upper Age,0 Through Upper Age,Petition,Non-petition,Petition,Non-petition,Petition,Non-petition\n')

for i in years:
    #print(i)
    in_file = "{}{}.csv".format(input_file_template,i)
    print('opening{}'.format(in_file))
    inf = open(in_file,'r')
    state = 0
    prev_state=0
    for line in inf:
            line_number += 1
            #determine state
            prev_state = state
            if 'Year' not in line and state==0:
                state=1
            elif len(line.split(','))<3 and state==1:
                state=2

            #state work
            #print('[{}]{}:{}'.format(state,line_number,line))
            if state==1:#write to combined
                outf.write(line)
    inf.close()
outf.close()
    

openingcleaning/clean1-97.csv
openingcleaning/clean1-98.csv
openingcleaning/clean1-99.csv
openingcleaning/clean1-00.csv
openingcleaning/clean1-01.csv
openingcleaning/clean1-02.csv
openingcleaning/clean1-03.csv
openingcleaning/clean1-09.csv
openingcleaning/clean1-10.csv
openingcleaning/clean1-11.csv
openingcleaning/clean1-12.csv
openingcleaning/clean1-13.csv
openingcleaning/clean1-15.csv
